In [62]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain.memory import ConversationBufferMemory



from threading import Thread
import warnings
warnings.filterwarnings("ignore")

In [63]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            device_map='auto',
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            # temperature=0.1
            )
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')

    def _call(self, prompt: str,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # print('_call函数内查看prompt', prompt)
        response, history = self.model.chat(self.tokenizer, prompt, history=[])
        # response, history = self.model.generate(self.tokenizer, prompt, history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [64]:
def init_model():
    # 初始化模型
    model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    # model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    return llm

In [65]:
def init_db():
    persist_directory = r'vectordb/chroma'
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2'
    # 加载词向量模型
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_model_cache_path)
    # 加载缓存知识库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
    )
    return vectordb

In [66]:
def intention_recognition(question: str, llm):
    template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断输入文本是否存在实体。
        2-抽取输入文本所有实体。
        3-根据实体与给出的意图标签进行判定该输入文本的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从输入的文本中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类输入文本。
        
        输入文本：{question}
        
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
            
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签

        有用的回答：
       """
          # 4. 意图分类：根据第3点的意图识别结果分类，意图标签：[政策知识，日常知识，招中标知识]。       2.意图识别：[意图1, 意图2, ...]。
    # ，例如日常、政策、设备、建筑、维修等
    response = llm.invoke(input=template.format(question))
    target = response.split('意图分类：')[-1]
    # print('模型实体+意图结果：', response)
    # print('意图分类:', target)
    return target
        

In [67]:
def qa_chain(vectordb, llm):
    """
    构建问答链
    :param persist_directory: 知识库本地保存路径，这里初始化政策信息知识库
    :return: 返回调用LLM回答
    """
    template = """
                使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说”谢谢你的提问！”。
                {context}
                问题：{question}
                有用的回答：
                """
    # 尽量使答案简明扼要。
    # 调用 LangChain 的方法来实例化一个 Template 对象，该对象包含了 context 和 question 两个变量，
    # 在实际调用时，这两个变量会被检索到的文档片段和用户提问填充
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=['context', 'question'], template=template)

    # 构造检索问答链
    qa_chain_ = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(),
        return_source_documents=True,
        chain_type_kwargs={'prompt': QA_CHAIN_PROMPT},
        # chain_type='stuff'
    )

    return qa_chain_


In [68]:
# 链接前面的函数
class Model_center():
    """
      存储问答 Chain 的对象
    """

    def __init__(self, ):
        print('初始化模型+知识库。。。。')
        self.model = init_model()
        self.db = init_db()
        

    def qa_chain_self_answer(self, question: str, chat_history: list = []):
        # print('调用问答链')
        # print('打印用户问题', question)
        if question == None or len(question) < 1:
            print('问答为空。。。。')
            return '', chat_history
        print('实体提取+意图识别。。。。')

        # for question in sentence:
        try:
            target = intention_recognition(question, self.model)
            print('意图识别成功。。。。')
        except:
            print('实体提取+意图识别步骤失败。。。。')
            
        if target == '招中标知识' or target == '政策知识':
            print('调用检索问答链。。。。')
            # print('知识库:', target)
            response = qa_chain(self.db, self.model).invoke({'query': question})['result']
        else:
            print('调用简单模型。。。。')
            template = f"""
                        回答输入的问题，如果你不知道答案，就说你不知道，不要试图编造答案。
                        输入的问题：{question}
                        有用的回答：
                        """
            response = self.model.invoke(input=template.format(question))
        print(f'意图分类：{target}，测试问题：{question}，返回结果：{response}')
        # try:
        
        # 结果调用下流式输出    target == '日常知识':
        
        
        # 检索问答链+历史聊天组件
        # response = self.qa_chain.invoke({'query': question, 'chat_history': chat_history})['answer']

        chat_history.append([question, response])
        # print(chat_history)
        return '', chat_history
                
        # except Exception as e:
        #     print('问答链报错', e)
        #     return e, chat_history

    def clear_history(self):
        self.qa_chain.clear_history()

In [69]:
model_center = Model_center()
def update_chatbot(question, chat_history):
    for char in model_center.qa_chain_self_answer(question, chat_history):
        gr.update(value=chat_history)
        chat_history.append((question, char))
    return chat_history
# def demo():
block = gr.Blocks()
with block as demo:
    with gr.Row(equal_height=True):  # 水平排列子组件
        with gr.Column(scale=15):  # 垂直排列子组件
            gr.Markdown("""<h1><center>QwenLM7B-Chat</center></h1><center>科大讯飞实践-招中标政策智能问答助手</center>""")

    with gr.Row():
        with gr.Column(scale=4):
            # 创建聊天界面的组件。height=450 参数设置了聊天界面的高度为 450 像素。
            # show_copy_button=True参数表示在聊天界面中显示一个复制按钮，允许用户复制聊天内容
            chatbot = gr.Chatbot(height=450, show_copy_button=True)
            # 创建一个文本框组件，用于输入 prompts。
            msg = gr.Textbox(label='Prompt/问题')

            with gr.Row():
                # 创建提交按钮
                db_wo_his_btn = gr.Button('Chat')
            with gr.Row():
                # 创建一个清除按钮，用于清除聊天机器人组件的内容。
                clear_btn = gr.ClearButton(components=[chatbot], value='Clear console')

        # 设置按钮的点击事件。当点击时，调用上面定义的 qa_chain_self_answer 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
        print('进度1')
        # 设置流式输出
        def bot(question, history):
            # print('bot_question',question)
            # print('bot_history',history)
            curr, response = model_center.qa_chain_self_answer(question, history)
            # print('response', response)
            # print('curr', curr)
            history = response
            bot_message = history[-1][1]
            # print('bot_message', bot_message)
            history[-1][1] = ''
            for character in bot_message:
                history[-1][1] += character
                # print(f'累计中：{history}')
                time.sleep(0.1)
                yield '', history
        
        db_wo_his_btn.click(bot, inputs=[msg, chatbot], outputs=[msg, chatbot])

        print('进度2')
        # 点击后清空后端存储的聊天记录
        clear_btn.click(model_center.clear_history)

    # 填写注意事项
    gr.Markdown(
        """
        提醒：<br>
        1. 初始化数据库实践可能较长，请耐心等待。
        2. 使用中如果出现异常，将会在文本输入框进行展示，请不要惊慌。 <br>
        """
    )
# gr.close_all()
# 直接启动
demo.queue()
demo.launch(server_name='127.0.0.1', server_port=6018)

初始化模型+知识库。。。。
正从本地加载模型。。。。。


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The repository for autodl-tmp/Qwen/Qwen-7B-Chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/autodl-tmp/Qwen/Qwen-7B-Chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for autodl-tmp/Qwen/Qwen-7B-Chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/autodl-tmp/Qwen/Qwen-7B-Chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


模型加载完成！
进度1
进度2
Running on local URL:  http://127.0.0.1:6018

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
实体提取+意图识别。。。。
意图识别成功。。。。
调用简单模型。。。。
意图分类：日常知识，测试问题：你好，返回结果：你好！有什么我能帮你的吗？
实体提取+意图识别。。。。
意图识别成功。。。。
调用简单模型。。。。
意图分类：日常知识，测试问题：你是谁？，返回结果：我是通义千问，由阿里云开发的预训练语言(si)模型。我被设计用来回答各种问题、提供信息和与用户进行对话。如果您有任何问题，请随时问我！
实体提取+意图识别。。。。
意图识别成功。。。。
调用简单模型。。。。
意图分类：日常知识，测试问题：周杰伦的父亲是谁？，返回结果：周杰伦的父亲是周耀中。
实体提取+意图识别。。。。
意图识别成功。。。。
调用检索问答链。。。。
意图分类：政策知识，测试问题：我想了解上海市奉贤区农村集体产权交易规则的具体内容，你知道吗？，返回结果：很抱歉，我没有找到关于上海市奉贤区农村集体产权交易规则的具体内容的信息。您可以在相关的官方网站或文件中查找相关信息，或者咨询当地的政府部门。
实体提取+意图识别。。。。
意图识别成功。。。。
调用检索问答链。。。。
意图分类：政策知识，测试问题：上海联合产权交易所有限公司物权交易操作指引的具体要求你知道吗，返回结果：是的，我知道上海联合产权交易所有限公司物促成权交易操作指引的具体要求。该指引是为了规范在联交所开展的物权交易活动，进一步明确针对物权交易特性的工作流程和管理操作标准，更好地开展物权交易市场化运作，发现资产价格，根据相关规定制定的。
实体提取+意图识别。。。。
意图识别成功。。。。
调用检索问答链。。。。
意图分类：政策知识，测试问题：请展示《上海联合产权交易所有限公司物权交易操作指引》的具体内容，返回结果：第一条为规范在上海联合产权交易所有限公司（以下简称“联交所”）开展的物权交易活动，进一步明确针对物权交易特性的工作流程和管理操作标准，更好地开展物权交易市场化运作，发现资产价格，根据相关规定制定本指引。

第二条在联交所内开展的物权交易是指《中华人民共和国物权法》规定的物的所有权以及用益物权的转让。法律、法规、规

In [ ]:
sentence = [
            '你好',
            '你是谁？',
            '上海联合产权交易所有限公司物权交易操作指引(沪联产交〔2020〕26号)',
            '周杰伦是谁？',
            '产权交易争议调解',
            '2023年7月，中国航天科技集团公司五院研制的长征十二号运载火箭在酒泉卫星发射中心成功发射，将一颗北斗导航卫星送入预定轨道。',
            '周杰伦的歌曲《稻香》是谁写的？',
            '项目名称为ABC，中标金额为500万元，供应商为XYZ公司。']

In [ ]:
template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断输入文本是否存在实体。
        2-抽取输入文本所有实体。
        3-根据实体与给出的意图标签进行判定该输入文本的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从输入的文本中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类输入文本。
        
        输入文本：{question}
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签

        有用的回答：
       """